<a href="https://colab.research.google.com/github/MJMortensonWarwick/AI-DL/blob/main/6_2_serving_a_model_as_an_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serving a Model as an API Endpoint
Here we'll serve our original model over an API. We'll start by rebuilding the model:

In [11]:
import tensorflow as tf

# the code from the last module
from sklearn.datasets import load_diabetes

import pandas as pd
import numpy as np

# import the data
data = load_diabetes()

tf_dataset = tf.data.Dataset.from_tensor_slices((data.data, data.target))

train_dataset = tf_dataset.take(400)
test_dataset = tf_dataset.skip(400)

train_batch = train_dataset.batch(50)
features, labels = next(iter(train_batch))

model = tf.keras.Sequential([
  tf.keras.layers.Dense(20, activation=tf.nn.relu, input_shape=(10, )),  # 10 features
  tf.keras.layers.Dense(20, activation=tf.nn.relu),
  tf.keras.layers.Dense(1)
])

# Keras implementation
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

model.fit(features, labels, epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 6ms/step - loss: 25657.6582 - mse: 25657.6582
Epoch 2/100
2/2 [==============================] - 0s 4ms/step - loss: 25653.5898 - mse: 25653.5898
Epoch 3/100
2/2 [==============================] - 0s 4ms/step - loss: 25649.5293 - mse: 25649.5293
Epoch 4/100
2/2 [==============================] - 0s 6ms/step - loss: 25645.0547 - mse: 25645.0547
Epoch 5/100
2/2 [==============================] - 0s 5ms/step - loss: 25640.4805 - mse: 25640.4805
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 25635.6680 - mse: 25635.6680
Epoch 7/100
2/2 [==============================] - 0s 5ms/step - loss: 25630.6699 - mse: 25630.6699
Epoch 8/100
2/2 [==============================] - 0s 6ms/step - loss: 25625.6621 - mse: 25625.6621
Epoch 9/100
2/2 [==============================] - 0s 6ms/step - loss: 25620.4004 - mse: 25620.4004
Epoch 10/100
2/2 [==============================] - 0s 6ms/step - loss: 25614.9199 - mse: 25614.9199

Now we will save the model to our local session store:

In [12]:
import pickle

with open("ann.pkl", 'wb') as file:  
    pickle.dump(model, file)

INFO:tensorflow:Assets written to: ram://7e644ee1-bde6-4338-a9c1-24301cbe2be5/assets


We're serving via FastAPI as we have previously. We need to setup our features as a class:

In [ ]:
!pip install fastapi[all]

import pydantic

from pydantic import BaseModel

class Features(BaseModel):
    age: float 
    sex: float 
    bmi: float 
    bp: float 
    s1: float 
    s2: float 
    s3: float 
    s4: float 
    s5: float 
    s6: float 

Next we can build our API. Refer back to the earlier API tutorial if the below in unclear

In [14]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("ann.pkl", "rb"))

@app.post('/predict')
def get_music_category(data: Features):
    received = data.dict()
    age = received['age']
    sex = received['sex']
    bmi = received['bmi']
    bp = received['bp']
    s1 = received['s1']
    s2 = received['s2']
    s3 = received['s3']
    s4 = received['s4']
    s5 = received['s5']
    s6 = received['s6']
    tf_dataset = tf.data.Dataset.from_tensor_slices([
                                          age, sex, bmi, bp,
                                          s1, s2, s3,
                                          s4, s5, s6])
    test_batch = tf_dataset.batch(1) # the whole dataset
    y_pred = model.predict(test_batch)
    return {'prediction': y_pred}

Lastly we will use the _colabcode_ package to turn our Colab into an API. In practice we can just use FastAPI directly

In [ ]:
!pip install colabcode

from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [16]:
server.run_app(app=app)

Public URL: NgrokTunnel: "https://1df4-34-125-223-241.ngrok.io" -> "http://localhost:10000"
